In [1]:
def image_latlon_pxpy(local_folder, X, Y, latitude, longitude):
    X = str(X)
    Y = str(Y)
    fname = f"{local_folder}/{X}/{Y}/{X}X{Y}Y_FINAL.tif"
    #print(fname, os.path.exists(fname))
    if os.path.exists(fname):
        dataset = rs.open(fname)
        px = longitude
        py = latitude
        px_pc = (px - dataset.bounds.left) / (dataset.bounds.right - dataset.bounds.left)
        py_pc = (dataset.bounds.top - py) / (dataset.bounds.top - dataset.bounds.bottom)
        return (np.floor(px_pc*dataset.width), np.floor(py_pc*dataset.height)), dataset.height, dataset.width
    else:
        return (0, 0), 0, 0

def check_ard_availability(local_folder, train_df, deploy_df, output_df, pixel_dist=16):
    '''
    Given a plot level CEO csv (one row per plot), identifies where ARD is not available.
    Identifies available data by calculating whether plot centroids are 
    >16 pixels away from a TTC tile edge.
    Requires TTC final tifs to be stored locally for image_latlon_pxpy()
    The output csv contains new columns X_tile and Y_tile which indicate
    the TTC tile id. Where the row contains 0 are areas where ARD is not
    available. 
    '''
    train_df = pd.read_csv(train_df)
    deploy_df = pd.read_csv(deploy_df) 
    
    train_df['X_tile'] = 0
    train_df['Y_tile'] = 0
    train_df['X_px'] = 0
    train_df['Y_px'] = 0
    train_df['X'] = 0.
    train_df['Y'] = 0.
    
    deploy_lons = deploy_df.X
    deploy_lats = deploy_df.Y
    
    for i, val in train_df[:100].iterrows():
        #try:
        lon = val.lon
        lat = val.lat
        x_tile = np.argmin(abs(lon - deploy_lons))
        lon_tile = deploy_df.X[x_tile]
        x_tile = deploy_df.X_tile[x_tile]
        
        y_tile = np.argmin(abs(lat - deploy_lats))
        lat_tile = deploy_df.Y[y_tile]
        y_tile = deploy_df.Y_tile[y_tile]
        
        l, w, h = image_latlon_pxpy(local_folder, x_tile, y_tile, lat, lon)
        #print(l, w, h)
        if l[0] > pixel_dist and l[1] > pixel_dist:
            if l[0] < (w - pixel_dist) and l[1] < (h - pixel_dist): 
                train_df.iloc[i, train_df.columns.get_loc('X_tile')] = x_tile
                train_df.iloc[i, train_df.columns.get_loc('Y_tile')] = y_tile

                train_df.iloc[i, train_df.columns.get_loc('X')] = lon_tile
                train_df.iloc[i, train_df.columns.get_loc('Y')] = lat_tile

                train_df.iloc[i, train_df.columns.get_loc('X_px')] = l[0]
                train_df.iloc[i, train_df.columns.get_loc('Y_px')] = l[1]
        else:
            print("skipping", l, w, h)
        #except:
        #    continue
            
    # drop rows where ARD is not available
    train_df = train_df[(train_df.X_tile != 0) & (train_df.Y_tile != 0)]
    train_df.to_csv(output_df, index = False)
    
    return None

In [7]:
import pandas as pd
import numpy as np
import rasterio as rs
import os
check_ard_availability(local_folder = '/Volumes/John/tof-output-2020/', deploy_df = '../../src/process_area_2022.csv',
                       train_df = 'ceo-plantations-train-v23-plot.csv', output_df = 'ceo-plantations-train-v23-plot-output.csv',
                       pixel_dist=16)

skipping (446.0, -1.0) 618 614
skipping (8.0, 194.0) 618 612
skipping (2.0, 265.0) 618 610
